In [1]:
import os, sys
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

In [2]:
fname='/expanse/nfs/cw3e/cwp140/preprocessed/PRISM/PRISM_HUC8_CO.nc'
PRISM = xr.open_dataset(fname)
PRISM = PRISM.sel(date=slice('2000-01-01', '2023-12-31'))

## get lat and lons of centroid
x = xr.DataArray(360+PRISM['lon'].values, dims=['HUC8'])
y = xr.DataArray(PRISM['lat'].values, dims=['HUC8'])

In [3]:
ds_lst = []
for i, year in enumerate(range(2000, 2024)):
    print(year)
    fname = '/expanse/nfs/cw3e/cwp140/preprocessed/ERA5/surface_pressure/{0}_sp.nc'.format(year)
    ds = xr.open_dataset(fname)
    ds = ds.sel(time=ds.time.dt.hour == 0) # select the 00 UTC hour
    ds = ds.sel(longitude=x, latitude=y, method='nearest')
    ds_lst.append(ds)
ds_lst[0]

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


<xarray.Dataset>
Dimensions:    (HUC8: 92, time: 366)
Coordinates:
    latitude   (HUC8) float32 37.25 36.75 38.75 38.5 ... 37.5 38.5 38.0 39.75
    longitude  (HUC8) float32 251.5 251.5 251.8 252.8 ... 258.2 258.5 258.5
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2000-12-31
Dimensions without coordinates: HUC8
Data variables:
    sp         (time, HUC8) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      Tue Apr  2 11:19:28 2024: ncks -O --mk_rec_dmn time /data/p...
    NCO:          netCDF Operators version 5.2.2 (Homepage = http://nco.sf.ne...

In [4]:
%%time
ds_final = xr.concat(ds_lst, dim="time")
ds_final

/home/dnash/miniconda3/envs/eaton-scripps-CO/lib/python3.12/site-packages/xarray/core/concat.py:527: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


CPU times: user 840 ms, sys: 2.42 s, total: 3.26 s
Wall time: 37.8 s


<xarray.Dataset>
Dimensions:    (HUC8: 92, time: 8766)
Coordinates:
    latitude   (HUC8) float32 37.25 36.75 38.75 38.5 ... 37.5 38.5 38.0 39.75
    longitude  (HUC8) float32 251.5 251.5 251.8 252.8 ... 258.2 258.5 258.5
  * time       (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2023-12-31
Dimensions without coordinates: HUC8
Data variables:
    sp         (time, HUC8) float32 7.908e+04 8.297e+04 ... 9.02e+04 9.006e+04
Attributes:
    Conventions:  CF-1.6
    history:      Tue Apr  2 11:19:28 2024: ncks -O --mk_rec_dmn time /data/p...
    NCO:          netCDF Operators version 5.2.2 (Homepage = http://nco.sf.ne...

In [13]:
## attach to PRISM dataset and convert to hPa
PRISM = PRISM.assign(sp=(['date','HUC8'], ds_final['sp'].values / 100.))
PRISM


<xarray.Dataset>
Dimensions:  (date: 8766, HUC8: 92)
Coordinates:
  * date     (date) datetime64[ns] 2000-01-01 2000-01-02 ... 2023-12-31
  * HUC8     (HUC8) object '14080107' '14080105' ... '11030001' '10250013'
Data variables:
    prec     (HUC8, date) float64 ...
    extreme  (HUC8, date) int64 ...
    lat      (HUC8) float64 37.18 36.86 38.77 38.38 ... 37.6 38.48 38.08 39.65
    lon      (HUC8) float64 -108.5 -108.6 -108.3 -107.3 ... -101.7 -101.4 -101.6
    sp       (date, HUC8) float32 790.8 829.7 795.9 720.5 ... 891.2 902.0 900.6

In [16]:
## function to add vars - sp_50, sp_100
PRISM = PRISM.assign(sp_50=(['date','HUC8'], (ds_final['sp'].values / 100.) - 50))
PRISM = PRISM.assign(sp_100=(['date','HUC8'], (ds_final['sp'].values / 100.) - 100))
PRISM

<xarray.Dataset>
Dimensions:  (date: 8766, HUC8: 92)
Coordinates:
  * date     (date) datetime64[ns] 2000-01-01 2000-01-02 ... 2023-12-31
  * HUC8     (HUC8) object '14080107' '14080105' ... '11030001' '10250013'
Data variables:
    prec     (HUC8, date) float64 ...
    extreme  (HUC8, date) int64 ...
    lat      (HUC8) float64 37.18 36.86 38.77 38.38 ... 37.6 38.48 38.08 39.65
    lon      (HUC8) float64 -108.5 -108.6 -108.3 -107.3 ... -101.7 -101.4 -101.6
    sp       (date, HUC8) float32 790.8 829.7 795.9 720.5 ... 891.2 902.0 900.6
    sp_50    (date, HUC8) float32 740.8 779.7 745.9 670.5 ... 841.2 852.0 850.6
    sp_100   (date, HUC8) float32 690.8 729.7 695.9 620.5 ... 791.2 802.0 800.6

In [ ]:
sp_start = np.empty(shape=(PRISM.date.size, PRISM.HUC8.size))
lev_lst = [1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 450, 400, 350, 300, 250, 225, 200, 175, 150, 125, 100, 70, 50, 30, 20, 10, 7, 5, 3, 2, 1]
for i, date in enumerate(PRISM.date.values):
    for j, HUC8 in enumerate(PRISM.HUC8.values):
        a = PRISM['sp_100'].sel(date=date, HUC8=HUC8).values
        b = PRISM['sp_50'].sel(date=date, HUC8=HUC8).values
        c = [x for x in lev_lst if a < x < b]
        # print(c[0])
        sp_start[i, j] = c[0]

700
775
700
650
700
650
750
700
700
775
775
800
825
775
775
800
825
825
700
700
700
700
700
775
825
750
700
700
700
775
750
775
750
825
700
750
700
750
700
700
800
775
650
775
800
825
825
825
650
650
650
600
650
700
650
700
775
800
825
650
700
700
700
750
800
600
700
650
825
800
700
650
700
650
650
650
750
650
700
800
825
825
800
700
700
775
600
700
800
825
825
825
700
775
700
650
700
650
700
700
700
775
750
775
800
775
775
800
825
825
700
700
700
700
700
775
825
750
700
700
700
750
750
775
750
800
700
700
700
750
700
700
775
750
650
750
800
825
800
825
650
650
650
600
600
700
650
700
750
800
825
650
700
650
700
750
775
600
700
650
800
800
700
600
700
650
650
650
750
650
700
800
825
800
775
700
700
775
600
700
775
825
825
825
700
750
700
650
700
650
700
700
700
775
750
775
800
775
775
800
825
825
700
700
700
700
700
775
825
750
700
700
700
750
750
775
750
800
700
700
700
750
700
700
775
750
650
750
800
825
800
825
650
650
650
600
600
700
650
700
750
800
825
650
700
650
700
750
775
600


In [ ]:
PRISM = PRISM.assign(sp_start=(['date','HUC8'], sp_start))
PRISM

In [38]:
PRISM

<xarray.Dataset>
Dimensions:   (date: 8766, HUC8: 92)
Coordinates:
  * date      (date) datetime64[ns] 2000-01-01 2000-01-02 ... 2023-12-31
  * HUC8      (HUC8) object '14080107' '14080105' ... '11030001' '10250013'
Data variables:
    prec      (HUC8, date) float64 ...
    extreme   (HUC8, date) int64 ...
    lat       (HUC8) float64 37.18 36.86 38.77 38.38 ... 37.6 38.48 38.08 39.65
    lon       (HUC8) float64 -108.5 -108.6 -108.3 ... -101.7 -101.4 -101.6
    sp        (date, HUC8) float32 790.8 829.7 795.9 720.5 ... 891.2 902.0 900.6
    sp_start  (date, HUC8) float64 700.0 775.0 700.0 650.0 ... 825.0 850.0 850.0

In [37]:
PRISM = PRISM.drop(['sp_100', 'sp_50'])

ValueError: One or more of the specified variables cannot be found in this dataset

In [39]:
## save file
fname = '/expanse/nfs/cw3e/cwp140/preprocessed/PRISM/PRISM_HUC8_CO_sp.nc'
PRISM.to_netcdf(path=fname, mode = 'w', format='NETCDF4')